# 📋 DAVID: 스마트 할 일 관리 앱 with 시각화
완료/삭제 버튼 색상, 완료 스타일 적용, 태그 필터, 마감일 정렬, 전체 평균 진행률, 꺾은선 그래프 포함

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import json
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

todo_items = []
progress_history = {}

# 진행률 바 + 라벨
overall_progress_bar = widgets.FloatProgress(min=0, max=100, value=0, bar_style='info', layout=widgets.Layout(width='300px'))
overall_progress_label = widgets.Label(value='(0%)')

tag_filter_dropdown = widgets.Dropdown(options=["전체"], value="전체", description="태그 필터:", layout=widgets.Layout(width='300px'))
todo_list_box = widgets.VBox(layout=widgets.Layout(border='1px solid #ccc', padding='10px', width='650px'))

def update_overall_progress():
    if not todo_items:
        overall_progress_bar.value = 0
        overall_progress_label.value = "(0%)"
        return
    avg = sum([i['progress'] for i in todo_items]) / len(todo_items)
    overall_progress_bar.value = avg
    overall_progress_label.value = f"({int(avg)}%)"
    today = datetime.now().strftime('%Y-%m-%d')
    progress_history[today] = int(avg)

def refresh_list():
    todo_list_box.children = []
    filtered = [i for i in todo_items if tag_filter_dropdown.value in ['전체', i['tag']]]
    sorted_items = sorted(filtered, key=lambda x: x['due'] or datetime.max)
    rows = []
    for item in sorted_items:
        style = {'background': '#f0f0f0', 'color': '#555'} if item['progress'] == 100 else {}
        title = widgets.Text(value=item['title'], layout=widgets.Layout(width='180px'))
        progress_bar = widgets.FloatProgress(value=item['progress'], min=0, max=100, layout=widgets.Layout(width='150px'))
        progress_input = widgets.BoundedIntText(value=item['progress'], min=0, max=100, layout=widgets.Layout(width='60px'))
        due = widgets.Label(value=item['due'].strftime('%Y-%m-%d') if item['due'] else '-', layout=widgets.Layout(width='80px'))
        tag = widgets.Label(value=f"#{item['tag']}" if item['tag'] else '', layout=widgets.Layout(width='80px'))
        complete_btn = widgets.Button(icon='check', button_style='success', layout=widgets.Layout(width='32px'))
        delete_btn = widgets.Button(icon='trash', button_style='danger', layout=widgets.Layout(width='32px'))

        def on_progress_change(change, i=item):
            i['progress'] = change['new']
            update_overall_progress()
            refresh_list()

        def on_delete(b, i=item):
            todo_items.remove(i)
            update_overall_progress()
            refresh_list()

        def on_complete(b, i=item):
            i['progress'] = 100
            update_overall_progress()
            refresh_list()

        progress_input.observe(lambda change, i=item: on_progress_change(change, i), names='value')
        delete_btn.on_click(lambda b, i=item: on_delete(b, i))
        complete_btn.on_click(lambda b, i=item: on_complete(b, i))

        row = widgets.HBox([title, progress_bar, progress_input, due, tag, complete_btn, delete_btn], layout=widgets.Layout(margin='10px 0'))
        if item['progress'] == 100:
            row.layout.border = '1px solid #aaa'
            title.style = {'text_color': '#666'}
        rows.append(row)
    todo_list_box.children = rows

# 입력 위젯
title_input = widgets.Text(placeholder='할 일을 입력하세요', layout=widgets.Layout(width='200px'))
progress_input = widgets.BoundedIntText(value=0, min=0, max=100, layout=widgets.Layout(width='80px'))
due_input = widgets.DatePicker(layout=widgets.Layout(width='200px'))
tag_input = widgets.Text(placeholder='예: 전공, 일상', layout=widgets.Layout(width='150px'))
add_button = widgets.Button(description='추가', button_style='success')

def add_task(b):
    if title_input.value.strip():
        todo_items.append({
            'title': title_input.value.strip(),
            'progress': progress_input.value,
            'due': due_input.value,
            'tag': tag_input.value.strip()
        })
        title_input.value = ''
        progress_input.value = 0
        due_input.value = None
        tag_input.value = ''
        update_overall_progress()
        tags = list(set(i['tag'] for i in todo_items if i['tag']))
        tag_filter_dropdown.options = ['전체'] + sorted(tags)
        refresh_list()

add_button.on_click(add_task)
tag_filter_dropdown.observe(lambda change: refresh_list(), names='value')

# 저장/불러오기
def save_data(b):
    with open('todo_data.json', 'w') as f:
        json.dump([{**i, 'due': i['due'].strftime('%Y-%m-%d') if i['due'] else None} for i in todo_items], f)

def load_data(b):
    try:
        with open('todo_data.json', 'r') as f:
            raw = json.load(f)
            todo_items.clear()
            for i in raw:
                todo_items.append({
                    'title': i['title'],
                    'progress': i['progress'],
                    'due': datetime.strptime(i['due'], '%Y-%m-%d').date() if i['due'] else None,
                    'tag': i['tag']
                })
            update_overall_progress()
            refresh_list()
    except: print("불러오기 실패")

# 그래프
    if not progress_history: return
    dates = sorted(progress_history.keys())[-7:]
    values = [progress_history[d] for d in dates]

save_btn = widgets.Button(description='저장', button_style='info')
load_btn = widgets.Button(description='불러오기', button_style='warning')
save_btn.on_click(save_data)
load_btn.on_click(load_data)

# 전체 출력
display(widgets.VBox([
    widgets.HBox([title_input, progress_input, due_input]),
    widgets.HBox([tag_input, add_button]),
    tag_filter_dropdown,
    widgets.HBox([overall_progress_bar, overall_progress_label]),
    todo_list_box
]))
update_overall_progress()
refresh_list()